# Import libraries

In [1]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
#from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score
from sklearn.preprocessing import MinMaxScaler

import torch
import torch.nn as nn
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import torch.optim as optim
import torch.nn.functional as F

import torchvision.transforms as transforms

import wandb
wandb.login()

import re
import os
import math
import random

import matplotlib.pyplot as plt

wandb: Currently logged in as: oe_sarmanova (use `wandb login --relogin` to force relogin)


# Dataset CNN

In [2]:
class IR_Dataset_DA(Dataset):
    def __init__(self, y_file, data_file, transform = None, target_transform = None):
        self.spec_labels = pd.read_csv(y_file, sep=',').iloc[:,1:]
        self.spec_number = pd.read_csv(y_file, sep=',').iloc[:,0]
        self.spec = pd.read_csv(data_file, sep=',').iloc[:,2:]
        self.domen = pd.read_csv(data_file, sep=',').iloc[:,1]
        self.transform = transform
        self.target_transform = target_transform

    def __len__(self):
        return len(self.spec_labels)

    def __getitem__(self, idx):
        label = torch.from_numpy(np.array(self.spec_labels.iloc[idx], dtype='float32'))
        number =  self.spec_number.iloc[idx]
        domen = self.domen.iloc[idx]
        sp = np.array(self.spec.iloc[(self.spec_number.iloc[idx]),:], dtype='float32')
        spec = torch.from_numpy(sp)
        #spec = torch.from_numpy(sp).unsqueeze(0)
        if self.transform:
            spec = self.transform(spec)
        if self.target_transform:
            label = self.target_transform(label)
        return number, spec, label, torch.from_numpy(np.array(domen, dtype='float32'))

# Feature Extractor

In [3]:
class FeatureExtractor(nn.Module):

    def __init__(self):
      
        super().__init__() # since Python 3.0

        self.layers = nn.Sequential(
            nn.Conv1d(in_channels=1, out_channels=2, kernel_size=9),
            nn.Sigmoid(),
            nn.MaxPool1d(2),
            nn.Conv1d(in_channels=2, out_channels=4, kernel_size=9),
            nn.Sigmoid(),
            nn.MaxPool1d(2),
            nn.Flatten(),
            nn.BatchNorm1d(461*4),
            nn.Linear(461*4, 64),
            nn.BatchNorm1d(64),
            nn.Sigmoid(),
            nn.Linear(64,32)
        )
    
    def forward(self, x):
        return self.layers(x)

In [4]:
class FeatureExtractor(nn.Module):
    def __init__(self):
      
        super().__init__() 

        self.layers = nn.Sequential(nn.Linear(1868,64),
                                    nn.Sigmoid(), 
                                    nn.Linear(64, 16))

    def forward(self, x):
        return self.layers(x)

# Classifier

In [5]:
class Classifier(nn.Module):
    def __init__(self):
      
        super().__init__() 

        self.layers = nn.Sequential(
            nn.BatchNorm1d(32),
            nn.Linear(32,8),
            nn.Sigmoid(),
            nn.BatchNorm1d(8),
            nn.Linear(8,1)
        ) 

    def forward(self, x):
        return self.layers(x)

In [6]:
class Classifier(nn.Module):
    def __init__(self):
      
        super().__init__() 

        self.layers = nn.Linear(16,1)

    def forward(self, x):
        return self.layers(x)


# Discriminator

In [7]:
class Discriminator(nn.Module):
    def __init__(self):
      
        super().__init__() 

        self.layers = nn.Sequential(
            nn.BatchNorm1d(32),
            nn.Linear(32,8),
            nn.Sigmoid(),
            nn.BatchNorm1d(8),
            nn.Linear(8,1),
            nn.Sigmoid()
        )

    def forward(self, x):
        return self.layers(x)

In [8]:
class Discriminator(nn.Module):
    def __init__(self):
      
        super().__init__() 

        self.layers = nn.Sequential(nn.Linear(16, 1),
                                    nn.Sigmoid())

    def forward(self, x):
        return self.layers(x)

In [9]:
def get_lambda(epoch, max_epoch):
    p = epoch / max_epoch
    return 2. / (1+np.exp(-10.*p)) - 1.

In [10]:
def S_sample_train(step, n_batches):
    global S_set_train
    if step % n_batches == 0:
        S_set_train = iter(S_train_dataloader)
    return next(S_set_train)

def S_sample_validation(step, n_batches):
    global S_set_validation
    if step % n_batches == 0:
        S_set_validation = iter(S_validation_dataloader)
    return next(S_set_validation)

# Utilities for output

In [11]:
def write_predictions_DANN(FeatExt, Classif, Discrim, param_name, split_path, val_loss, dataloader, feat_num, dset, dm):

    '''load feature extractor model'''
    checkpoint_f = torch.load(split_path + 'model'+model_name_f+'_vld_loss_'+str(int(round(val_loss,3)))+'.pth')
    FeatExt.load_state_dict(checkpoint_f['model_state_dict'])
    
    '''load classifier model'''
    checkpoint_c = torch.load(split_path + 'model'+model_name_c+'_vld_loss_'+str(int(round(val_loss,3)))+'.pth')
    Classif.load_state_dict(checkpoint_c['model_state_dict'])
    
    '''load discriminator model'''
    checkpoint_d = torch.load(split_path + 'model'+model_name_d+'_vld_loss_'+str(int(round(val_loss,3)))+'.pth')
    Discrim.load_state_dict(checkpoint_d['model_state_dict'])
    
    FeatExt.eval()
    Classif.eval()
    Discrim.eval()

    y_out, y_true = torch.zeros(1, feat_num+1), torch.zeros(1, feat_num+1)
    d_out, d_true = torch.zeros(1, feat_num+1), torch.zeros(1, feat_num+1)
    
    for num, specs, labels, domain in dataloader:
        y_outputs = Classif(FeatExt(specs))
        y_outputs[y_outputs<0]=0
        
        y_out = torch.cat((y_out, torch.cat((num.unsqueeze(-1), y_outputs), -1)),0)
        y_true = torch.cat((y_true, torch.cat((num.unsqueeze(-1), labels), -1)),0)
        
        d_outputs = Discrim(FeatExt(specs))
        
        d_out = torch.cat((d_out, torch.cat((num.unsqueeze(-1), d_outputs), -1)),0)
        d_true = torch.cat((d_true, torch.cat((num.unsqueeze(-1), domain.unsqueeze(-1)), -1)),0)
    
    a = ['pattern_number',param_name +'_true','pattern_number', param_name+'_predicted']

    pd.concat([pd.DataFrame(y_true.detach().numpy()), pd.DataFrame(y_out.detach().numpy())], axis = 1).to_csv(split_path + 'conc_'+dset+'_'+dm+'.csv',sep=',', header = a)
    pd.concat([pd.DataFrame(d_true.detach().numpy()), pd.DataFrame(d_out.detach().numpy())], axis = 1).to_csv(split_path + 'domain_'+dset+'_'+dm+'.csv',sep=',', header = a)

In [12]:
def clear_folder(folder, model_name, loss_value):
    for filename in os.listdir(folder):
        pat = r'.+\.pth'
        if filename[len('model'):(len('model')+len(model_name))] == model_name:
            if re.match(pat, filename):
                letters = len('model'+model_name+'_vld_loss_')
                if int(filename[letters:-4])!=int(loss_value):os.remove(folder+filename)
                else:
                    continue

In [13]:
def reset_weights(m):
    '''
    Reset model weights.
    '''
    for layer in m.children():
        if hasattr(layer, 'reset_parameters'):
            print(f'Reset trainable parameters of layer = {layer}')
            layer.reset_parameters()

# Utilities for feature processing

In [14]:
class MinMax_Normalize:
    def __init__(self,feat_min,feat_max):
        assert feat_min.shape == feat_max.shape
        self.min = feat_min
        self.max = feat_max

    def __call__(self,x):
        # x of shape [batch_size, features]
        #assert x.shape[1] == self.min.shape[0]
        return (x - self.min) / (self.max - self.min)

In [15]:
#Calculate min and max
def min_max(train_dataloader):
    data = next(iter(train_dataloader))
    MIN = torch.min(data[1],0).values
    MAX = torch.max(data[1],0).values

    return MIN, MAX

# Hyper parameters

In [16]:
input_num = 1868
outs_num = 1
#neurons_num = 128 # neurons in  hidden layer

batch_size = 64

trn_batch_s = 1203
vld_batch_s = 343
tst_batch_s = 173

trn_batch_t = 347
vld_batch_t = 99
tst_batch_t = 50

lamda = 0.1
l2_lambda = 0.001
test_stop = 100        #stopping criterion is an error on the validation set
                        #we stop if the error on the validation set (val_loss) did not fall within 250 epochs (test_stop)
INITIALS = 5 # number of weight's initializations

k_folds = [[42,12],[612,45],[72,172],[871,48],[52,134],[139,15],[287,403],[32,29]]

# Paths

In [17]:
gen_path ='C:\\Users\\helga\\OneDrive\\Рабочий стол\\Lab\\2024\\wines_da\\'


x_source = gen_path + 'data\\spectra_source.csv'
y_source = pd.read_csv(gen_path + 'data\\conc_source.csv')

x_target = gen_path + 'data\\spectra_target.csv'
y_target = pd.read_csv(gen_path + 'data\\conc_target.csv')

param = 'ethanol_pct'
k = 1

model_name_f = '_MLP_N02_64_16_f'
model_name_c = '_MLP_N01_16_c'
model_name_d = '_MLP_N01_16_d'

case_name = 'DANN_ethanol_test_stop_'+str(test_stop)+ model_name_f + '_batch_64_19'
case_path = gen_path + case_name+'/'

In [18]:
'''models'''
feat_ext = FeatureExtractor()
clsf = Classifier()
dscr = Discriminator()

'''loss functions'''
loss_function = torch.nn.MSELoss() 
bce = nn.BCELoss()

'''optimizers'''
f_opt = torch.optim.Adam(feat_ext.parameters(), lr=0.001)
c_opt = torch.optim.Adam(clsf.parameters(), lr=0.001)
d_opt = torch.optim.Adam(dscr.parameters(), lr=0.001)


In [19]:
for fold in k_folds:
    
    split_path = case_path+'split_'+ str(fold[0])+'_'+str(fold[1])+ '/'
    os.makedirs(split_path, exist_ok=True)

    '''Split and write Y data'''
    Y_trn_s, Y_30_s = train_test_split(y_source.iloc[:,[0,k]], test_size=0.3, random_state=fold[0])
    Y_vld_s, Y_tst_s = train_test_split(Y_30_s, test_size = 0.3333, random_state=fold[1])
    
    Y_trn_t, Y_30_t = train_test_split(y_target.iloc[:,[0,k]], test_size=0.3, random_state=fold[0])
    Y_vld_t, Y_tst_t = train_test_split(Y_30_t, test_size = 0.3333, random_state=fold[1])
    
    a = ['sample_number',param]

    pd.DataFrame(Y_trn_s).to_csv(split_path + 'Y_trn_s.csv',sep=',', index=False, header = a)
    pd.DataFrame(Y_vld_s).to_csv(split_path + 'Y_vld_s.csv',sep=',', index=False, header = a)
    pd.DataFrame(Y_tst_s).to_csv(split_path + 'Y_tst_s.csv',sep=',', index=False, header = a)
    
    pd.DataFrame(Y_trn_t).to_csv(split_path + 'Y_trn_t.csv',sep=',', index=False, header = a)
    pd.DataFrame(Y_vld_t).to_csv(split_path + 'Y_vld_t.csv',sep=',', index=False, header = a)
    pd.DataFrame(Y_tst_t).to_csv(split_path + 'Y_tst_t.csv',sep=',', index=False, header = a)
    
    '''Import and normalization source data'''
    training_data_s = IR_Dataset_DA(split_path+'Y_trn_s.csv', x_source)
    train_dataloader_s = DataLoader(training_data_s, batch_size=trn_batch_s, shuffle=True)
    min_s, max_s = min_max(train_dataloader_s)
    
    trn_data_s = IR_Dataset_DA(split_path+'Y_trn_s.csv', x_source, transform = transforms.Compose([MinMax_Normalize(min_s,max_s)]))
    vld_data_s = IR_Dataset_DA(split_path+'Y_vld_s.csv', x_source, transform = transforms.Compose([MinMax_Normalize(min_s,max_s)]))
    tst_data_s = IR_Dataset_DA(split_path+'Y_tst_s.csv',x_source, transform = transforms.Compose([MinMax_Normalize(min_s,max_s)]))
    
    train_dataloader_s = DataLoader(trn_data_s, batch_size=64, shuffle=True)    
    vld_dataloader_s = DataLoader(vld_data_s, batch_size=64, shuffle=True)
    test_dataloader_s = DataLoader(tst_data_s, batch_size=64, shuffle=True)
    
    '''Import and normalization target data'''
    training_data_t = IR_Dataset_DA(split_path+'Y_trn_t.csv', x_target)
    train_dataloader_t = DataLoader(training_data_t, batch_size=trn_batch_t, shuffle=True)
    min_t, max_t = min_max(train_dataloader_t)
    
    trn_data_t = IR_Dataset_DA(split_path+'Y_trn_t.csv',x_target, transform = transforms.Compose([MinMax_Normalize(min_t,max_t)]))
    vld_data_t = IR_Dataset_DA(split_path+'Y_vld_t.csv',x_target, transform = transforms.Compose([MinMax_Normalize(min_t,max_t)]))
    tst_data_t = IR_Dataset_DA(split_path+'Y_tst_t.csv',x_target, transform = transforms.Compose([MinMax_Normalize(min_t,max_t)]))
    
    train_dataloader_t = DataLoader(trn_data_t, batch_size=19, shuffle=True)    
    vld_dataloader_t = DataLoader(vld_data_t, batch_size=19, shuffle=True)
    test_dataloader_t = DataLoader(tst_data_t, batch_size=19, shuffle=True)
         
    for init_number in range(0,5):
        
        min_val_loss = [1000000000000000.0] #just big random number to enter learning cycle
        wandb.init(project = case_name)

        init_path = split_path + str(init_number)+'/'
        os.makedirs(init_path, exist_ok=True)
        
        split_name = 'split_'+ str(fold[0])+'_'+str(fold[1])+'/'
        init_name = str(init_number) + '/'
        model_name = 'DANN'
        wandb.run.name = split_name + init_name + model_name
        wandb.run.save()
        
        for epoch_step in range(0, 5000, test_stop):
            best_epoch=0
            
            if epoch_step==0:
                feat_ext.apply(reset_weights)
                clsf.apply(reset_weights)
                dscr.apply(reset_weights)
                
                for ep in range(0, test_stop):
                    
                    s_iterator = iter(train_dataloader_s)
                    t_iterator = iter(train_dataloader_t)
                    
                    for _ in range(0, int(trn_batch_s/batch_size)+1): # get bacth
                        num_s, inputs_s, label_s, domain_s = next(s_iterator) # parse batch
                        num_t, inputs_t, label_t, domain_t = next(t_iterator) # parse batch
                        
                        x = torch.cat([inputs_s, inputs_t], dim=0)
                        labels = torch.cat([label_s, label_t], dim=0)
                        domain = torch.cat([domain_s, domain_t], dim=0).unsqueeze(-1)
                        
                        d_opt.zero_grad()
                        d_out = dscr(feat_ext(x))
                        dscr_loss = bce(d_out, domain)
                        dscr_loss.backward(retain_graph=True)
                        d_opt.step()
        
                        clsf_out = clsf(feat_ext(x))
                        d_out = dscr(feat_ext(x))
                        clsf_loss = loss_function(clsf_out, labels)
                        dscr_loss = bce(d_out, domain)
                        
                        total_loss = clsf_loss -lamda*dscr_loss
                        
                        d_opt.zero_grad()
                        c_opt.zero_grad()
                        f_opt.zero_grad()
                        
                        total_loss.backward()
                        f_opt.step()
                        c_opt.step()
                        
                    val_loss = 0.0
                    s_iterator = iter(vld_dataloader_s)
                    t_iterator = iter(vld_dataloader_t)
                    
                    for _ in range(0, int(vld_batch_s/batch_size)+1):
                        num_s, inputs_s, label_s, domain_s = next(s_iterator) # parse batch
                        num_t, inputs_t, label_t, domain_t = next(t_iterator) # parse batch
                        
                        x = torch.cat([inputs_s, inputs_t], dim=0)
                        labels = torch.cat([label_s, label_t], dim=0)
                        domain = torch.cat([domain_s, domain_t], dim=0).unsqueeze(-1)
                        d_out_val = dscr(feat_ext(x))
                        dscr_loss_val = bce(d_out_val, domain)
                        clsf_loss_val = loss_function(clsf(feat_ext(x)), labels)
                        
                        val_loss += clsf_loss_val -lamda*dscr_loss_val

                    wandb.log({"trn_loss": total_loss, "vld_loss": val_loss})
                    wandb.log({"clsf_trn_loss": clsf_loss, "clsf_vld_loss": clsf_loss_val})
                    wandb.log({"dscr_trn_loss": dscr_loss, "dscr_vld_loss": dscr_loss_val})
                    
                    if val_loss.item() <= min(min_val_loss):
                        torch.save({'epoch': ep,
                                    'model_state_dict': feat_ext.state_dict(),
                                    'optimizer_state_dict': f_opt.state_dict(),
                                    'loss': total_loss}, init_path + 'model'+model_name_f+'_vld_loss_'+str(int(round(val_loss.item(),3)))+'.pth')
                        
                        torch.save({'epoch': ep,
                                    'model_state_dict': clsf.state_dict(),
                                    'optimizer_state_dict': c_opt.state_dict(),
                                    'loss': clsf_loss}, init_path + 'model'+model_name_c+'_vld_loss_'+str(int(round(val_loss.item(),3)))+'.pth')
                        
                        torch.save({'epoch': ep,
                                    'model_state_dict': dscr.state_dict(),
                                    'optimizer_state_dict': d_opt.state_dict(),
                                    'loss': dscr_loss}, init_path + 'model'+model_name_d+'_vld_loss_'+str(int(round(val_loss.item(),3)))+'.pth')
    
                        min_val_loss.append(round(val_loss.item(),3))

                clear_folder(init_path,model_name_f,min(min_val_loss))
                clear_folder(init_path,model_name_c,min(min_val_loss))
                clear_folder(init_path,model_name_d,min(min_val_loss))
                min_val_loss = [min(min_val_loss)]
        
            if epoch_step!=0:
                
                checkpoint_f = torch.load(init_path + 'model'+model_name_f+'_vld_loss_'+str(int(min_val_loss[0]))+'.pth')
                feat_ext.load_state_dict(checkpoint_f['model_state_dict'])
                f_opt.load_state_dict(checkpoint_f['optimizer_state_dict'])
                last_best_epoch = checkpoint_f['epoch']
                total_loss = checkpoint_f['loss']
                feat_ext.train()
            
                checkpoint_c = torch.load(init_path + 'model'+model_name_c+'_vld_loss_'+str(int(min_val_loss[0]))+'.pth')
                clsf.load_state_dict(checkpoint_c['model_state_dict'])
                c_opt.load_state_dict(checkpoint_c['optimizer_state_dict'])
                last_best_epoch = checkpoint_c['epoch']
                clsf_loss = checkpoint_c['loss']
                clsf.train()

                checkpoint_d = torch.load(init_path + 'model'+model_name_d+'_vld_loss_'+str(int(min_val_loss[0]))+'.pth')
                dscr.load_state_dict(checkpoint_d['model_state_dict'])
                d_opt.load_state_dict(checkpoint_d['optimizer_state_dict'])
                last_best_epoch = checkpoint_d['epoch']
                dscr_loss = checkpoint_d['loss']
                dscr.train()
                
                
                if last_best_epoch + test_stop > ep:
                    for ep in range(epoch_step, epoch_step+test_stop):
                        s_iterator = iter(train_dataloader_s)
                        t_iterator = iter(train_dataloader_t)

                        for _ in range(0, int(trn_batch_s/batch_size)+1): # get bacth
                            num_s, inputs_s, label_s, domain_s = next(s_iterator) # parse batch
                            num_t, inputs_t, label_t, domain_t = next(t_iterator) # parse batch

                            x = torch.cat([inputs_s, inputs_t], dim=0)
                            labels = torch.cat([label_s, label_t], dim=0)
                            domain = torch.cat([domain_s, domain_t], dim=0).unsqueeze(-1)

                            d_opt.zero_grad()
                            d_out = dscr(feat_ext(x))
                            dscr_loss = bce(d_out, domain)
                            dscr_loss.backward(retain_graph=True)
                            d_opt.step()

                            clsf_out = clsf(feat_ext(x))
                            d_out = dscr(feat_ext(x))
                            clsf_loss = loss_function(clsf_out, labels)
                            dscr_loss = bce(d_out, domain)

                            total_loss = clsf_loss -lamda*dscr_loss

                            d_opt.zero_grad()
                            c_opt.zero_grad()
                            f_opt.zero_grad()

                            total_loss.backward()
                            f_opt.step()
                            c_opt.step()
                        
                        val_loss = 0.0
                        s_iterator = iter(vld_dataloader_s)
                        t_iterator = iter(vld_dataloader_t)
                        for _ in range(0, int(vld_batch_s/batch_size)+1):
                            num_s, inputs_s, label_s, domain_s = next(s_iterator) # parse batch
                            num_t, inputs_t, label_t, domain_t = next(t_iterator) # parse batch

                            x = torch.cat([inputs_s, inputs_t], dim=0)
                            labels = torch.cat([label_s, label_t], dim=0)
                            domain = torch.cat([domain_s, domain_t], dim=0).unsqueeze(-1)
                            d_out_val = dscr(feat_ext(x))

                            dscr_loss_val = bce(d_out_val, domain)
                            clsf_loss_val = loss_function(clsf(feat_ext(x)), labels)

                            val_loss += clsf_loss_val - lamda*dscr_loss_val

                        wandb.log({"trn_loss": total_loss, "vld_loss": val_loss})
                        wandb.log({"clsf_trn_loss": clsf_loss, "clsf_vld_loss": clsf_loss_val})
                        wandb.log({"dscr_trn_loss": dscr_loss, "dscr_vld_loss": dscr_loss_val})

                        if val_loss.item() <= min(min_val_loss):
                            torch.save({'epoch': ep,
                                        'model_state_dict': feat_ext.state_dict(),
                                        'optimizer_state_dict': f_opt.state_dict(),
                                        'loss': total_loss}, init_path + 'model'+model_name_f+'_vld_loss_'+str(int(round(val_loss.item(),3)))+'.pth')

                            torch.save({'epoch': ep,
                                        'model_state_dict': clsf.state_dict(),
                                        'optimizer_state_dict': c_opt.state_dict(),
                                        'loss': clsf_loss}, init_path + 'model'+model_name_c+'_vld_loss_'+str(int(round(val_loss.item(),3)))+'.pth')

                            torch.save({'epoch': ep,
                                        'model_state_dict': dscr.state_dict(),
                                        'optimizer_state_dict': d_opt.state_dict(),
                                        'loss': dscr_loss}, init_path + 'model'+model_name_d+'_vld_loss_'+str(int(round(val_loss.item(),3)))+'.pth')

                            min_val_loss.append(round(val_loss.item(),3))

                    clear_folder(init_path,model_name_f,min(min_val_loss))
                    clear_folder(init_path,model_name_c,min(min_val_loss))
                    clear_folder(init_path,model_name_d,min(min_val_loss))
                    min_val_loss = [min(min_val_loss)]
                
                else: continue 
                        
                        
        wandb.finish()

        write_predictions_DANN(feat_ext, clsf, dscr, param, init_path, min_val_loss[0], train_dataloader_s, outs_num, dset = 'trn', dm='s')
        write_predictions_DANN(feat_ext, clsf, dscr, param, init_path, min_val_loss[0], vld_dataloader_s, outs_num, dset = 'vld', dm='s')
        write_predictions_DANN(feat_ext, clsf, dscr, param, init_path, min_val_loss[0], test_dataloader_s, outs_num, dset = 'tst', dm='s')

        write_predictions_DANN(feat_ext, clsf, dscr, param, init_path, min_val_loss[0], train_dataloader_t, outs_num, dset = 'trn', dm='d')
        write_predictions_DANN(feat_ext, clsf, dscr, param, init_path, min_val_loss[0], vld_dataloader_t, outs_num, dset = 'vld', dm='d')
        write_predictions_DANN(feat_ext, clsf, dscr, param, init_path, min_val_loss[0], test_dataloader_t, outs_num, dset = 'tst', dm='d')

wandb: wandb version 0.17.0 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


wandb: WARNING Calling run.save without any arguments is deprecated.Changes to attributes are automatically persisted.


Reset trainable parameters of layer = Linear(in_features=1868, out_features=64, bias=True)
Reset trainable parameters of layer = Linear(in_features=64, out_features=16, bias=True)
Reset trainable parameters of layer = Linear(in_features=16, out_features=1, bias=True)
Reset trainable parameters of layer = Linear(in_features=16, out_features=1, bias=True)


trn_loss,0.15129
vld_loss,2.1951
_step,2699
_runtime,551
_timestamp,1716889431
clsf_trn_loss,0.18816
clsf_vld_loss,0.25817
dscr_trn_loss,0.36864
dscr_vld_loss,0.4382


trn_loss,█▄▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
vld_loss,█▄▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
_step,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
_runtime,▁▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇███
_timestamp,▁▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇███
clsf_trn_loss,█▄▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
clsf_vld_loss,█▄▂▁▁▁▁▁▁▁▃▁▁▁▁▁▁▁▁▁▂▁▁▁▁▂▁▁▁▁▁▁▂▁▁▁▁▂▁▁
dscr_trn_loss,▄▇▆▄▁▄▁▁▂▅▅▆▄▅▅▅▄▇▇▆▇▆▅▆▆▆▆▅▄▄▅█▃▂▄▅▃▂▄▆
dscr_vld_loss,▄▅▄▂▂▃▃▁▂▄▆▅▄▅▄▃▅▇▅▅▅▄▅▄▅▅█▄▃▃▄▇▃▃▃▄▆▂▃▄


...

wandb: wandb version 0.17.0 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


wandb: WARNING Calling run.save without any arguments is deprecated.Changes to attributes are automatically persisted.


Reset trainable parameters of layer = Linear(in_features=1868, out_features=64, bias=True)
Reset trainable parameters of layer = Linear(in_features=64, out_features=16, bias=True)
Reset trainable parameters of layer = Linear(in_features=16, out_features=1, bias=True)
Reset trainable parameters of layer = Linear(in_features=16, out_features=1, bias=True)


trn_loss,0.33154
vld_loss,3.57041
_step,1499
_runtime,342
_timestamp,1716909445
clsf_trn_loss,0.36789
clsf_vld_loss,0.64319
dscr_trn_loss,0.36348
dscr_vld_loss,0.43108


trn_loss,█▇▆▃▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
vld_loss,██▇▅▃▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
_step,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
_runtime,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
_timestamp,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
clsf_trn_loss,█▇▆▃▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
clsf_vld_loss,▇▇█▇▃▂▂▂▂▁▂▁▁▁▁▂▂▁▂▁▁▁▂▂▁▁▁▁▁▁▁▁▁▁▁▁▂▁▁▁
dscr_trn_loss,▄▄█▇▃▁▃▄▃▃▃▂▃▃▃▂▄▃▃▃▄▂▃▃▂▃▃▂▃▃▃▃▃▃▃▃▃▂▃▂
dscr_vld_loss,▃▃█▆▃▄▃▅▁▃▃▂▃▃▂▂▃▂▃▂▃▁▃▂▂▃▂▁▃▂▂▃▃▂▃▂▃▂▂▂
